# **Infer `Cellmask` and `Nucleus`**

***Prior to this notebook, you should have already run through [1.0_image_setup](1.0_image_setup.ipynb).***

### ➡️ **Input:**
In this workflow, a single or multi-channel confocal microscopy image of fluorescently tagged organelles will be used to "infer" (or segment) the cell and nucleus masks. The following setup is recommended for this pipeline:

| **Imaging Requirements**|[**Masks Workflow (D)**](./1.1d_infer_masks_from-composite_ipsc.ipynb)|
| :------------------------------------- |  :------:  |
| Nuclei Marker                          |      ✔     |
| Cell Membrane Marker                   |      ✔     |
| Cytoplasmic Organelles                 |      ✔     |
| Number of cells per image              |   Single or Multiple   |
| Applicable with sample data         |  IPSC   |

*If your images are not compatible with this setup, consider the other 1.1_Masks Workflows available:*

| **Imaging Requirements**|[**Masks Workflow**](./1.1_infer_masks_from-composite_with_nuc.ipynb)|[**Masks Workflow (A)**](./1.1a_infer_masks_from-composite_single_cell.ipynb)|[**Masks Workflow (B)**](./1.1b_infer_masks_from-composite_multiple-cells.ipynb)|[**Masks Workflow (C)**](./1.1c_infer_masks_from-composite_neuron_with_pm.ipynb)|
| :------------------------------------- |  :------:  |  :------:  |  :------:  |  :------:  |
| Nuclei Marker                          |     ✔     |      ✘     |     ✘     |     ✔     |
| Cell Membrane Marker                   |     ✘     |       ✘    |    ✘     |     ✔     |
| Cytoplasmic Organelles                 |     ✔     |      ✔     |     ✔     |     ✔     |
| Number of cells per image              |  Single or Multiple  |   Single   |  Single or Multiple |  Single or Multiple |
| Applicable with sample data         |  ✘  | Neuron_1   |  Astrocyte | Neuron_2|

### **Included in this Notebook:**
1. **Infer *nuclei*** - Segment the ***nuclei*** from a single channel (nuclei marker). This will be necessary to determine the other subcellular compartments. The nuclei will also be used to seed the instance segmentation of the ***cell*** area (***cellmask***).
    
    *Convention: *"nuclei"* for the segmentation of ALL nuclei in the image. *"nucleus"* for the single nucleus associated to the single cell being analyzed after the cell with the most signal is determined.*

2. **Infer *cellmask*** - Segment multiple ***cellmasks*** via a watershedding algorithm, then select the ***cellmask*** with the most aggregate signal. The **cellmask** will be necessary for selecting the ***nucleus***.
-----

### 👣 **Summary of steps**

➡️ **EXTRACTION**
- **`STEP 1`** - Create composite image

    - determine weight to apply to each channel of the intensity image (w# = user input)
    - select plasma membrane channel (pm_channel = user input)
    - choose whether to invert plasma membrane or not (invert_PM = user input)

- **`STEP 2`** - Segment nuclei (from label)

    - (A) select single channel containing the nuclei marker (channel number = user input)
    - (B) rescale intensity of composite image (min=0, max=1); apply median (median size = user input) and gaussian filter (sigma = user input)
    - (C) log transform image and calculate Li's minimum cross entropy threshold value; apply threshold to image (thresholding options = user input)
    - (D) fill holes (hole size = user input) and remove small objects (object size = user input)
    - (E) label nuclei objects
 
**CORE PROCESSING**
- **`STEP 3`** - Watershed for cellmask

    - select plasma membrane channel (pm_channel = user input)
    - perform water-shedding algorithm on composite using the nuclei as seeds (method = user input)

**POST-PROCESSING**
- **`STEP 4`** - Select cell from watershed

    - cell mask (and corresponding nucleus) is selected based on maximum raw signal from composite image

**POST-POST-PROCESSING**
- **`STEP 5`** - Hole Filling

    - fill holes (hole size = user input)
    - determine filling method (fill_2d = user input)

**OUTPUT** ➡️ 
- **`STEP 6`** - Stack layers

    - Stack masks in order of nucleus, then cellmask

---------------------
## **IMPORTS AND LOAD IMAGE**
Details about the functions included in this subsection are outlined in the [`1.0_image_setup`](1.0_image_setup.ipynb) notebook. Please visit that notebook first if you are confused about any of the code included here.

#### &#x1F3C3; **Run code; no user input required**

In [6]:
from pathlib import Path
import os

import numpy as np
import pandas as pd
import napari
from napari.utils.notebook_display import nbscreenshot

from infer_subc.core.file_io import (read_czi_image, 
                                     export_inferred_organelle,
                                     list_image_files,
                                     sample_input)
from infer_subc.core.img import *
from infer_subc.organelles.membrane import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### &#x1F6D1; &#x270D; **User Input Required:**

Please specify the following information about your data: `im_type`, `data_root_path`, `in_data_path`, and `out_data_path`.

In [7]:
### USER INPUT REQUIRED ###
# If using the sample data, set cell_type to "ipsc":
# If not using the sample data, set cell_type to None
sample_data_type = "ipsc"

# If you are not using the sample data, please edit "USER SPECIFIED" as necessary.
## Define the path to the directory that contains the input image folder.
data_root_path = Path("USER SPECIFIED")

# Specify the file type of your raw data that will be analyzed. Ex) ".czi" or ".tiff"
im_type = "USER SPECIFIED"

## Specify which subfolder that contains the input data and the input data file extension
in_data_path = data_root_path / "USER SPECIFIED"

## Specify the output folder to save the segmentation outputs if.
## If its not already created, the code below will creat it for you
out_data_path = data_root_path / "USER SPECIFIED"

#### &#x1F3C3; **Run code; no user input required**

In [8]:
# If sample_data_type is set "neuron_1", then the sample data is used and the directories are set
if sample_data_type != None:
    data_root_path, im_type, in_data_path, out_data_path = sample_input(sample_data_type)

# list files in the input folder
img_file_list = list_image_files(in_data_path,im_type)
pd.set_option('display.max_colwidth', None)
pd.DataFrame({"Image Name":img_file_list})

,Image Name
0,c:\Users\redre\Documents\CohenLab\scohen_lab_repo\infer-subc\sample_data\example_ipsc\raw\01202024_MSi08L_iPSCs_undiff_BR5_N14_Unmixing_0_cmle.ome.tiff


#### &#x1F6D1; &#x270D; **User Input Required:**

Use the list above to specify which image you wish to analyze based on its index: `test_img_n`

In [9]:
#### USER INPUT REQUIRED ###
test_img_n = 0

#### &#x1F3C3; **Run code; no user input required**

In [10]:
# load image and metadata
test_img_name = img_file_list[test_img_n]
img_data,meta_dict = read_czi_image(test_img_name)

# metadata
channel_names = meta_dict['name']
meta = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']
file_path = meta_dict['file_name']
print("Metadata information")
print(f"File path: {file_path}")
for i in list(range(len(channel_names))):
    print(f"Channel {i} name: {channel_names[i]}")
print(f"Scale (ZYX): {scale}")
print(f"Channel axis: {channel_axis}")

# open viewer and add images
viewer = napari.Viewer()
for i in list(range(len(channel_names))):
    viewer.add_image(img_data[i],
                     scale=scale,
                     name=f"Channel {i}")
viewer.grid.enabled = True
viewer.reset_view()
print("\nProceed to Napari window to view your selected image.")

Metadata information
File path: c:\Users\redre\Documents\CohenLab\scohen_lab_repo\infer-subc\sample_data\example_ipsc\raw\01202024_MSi08L_iPSCs_undiff_BR5_N14_Unmixing_0_cmle.ome.tiff
Channel 0 name: 01202024_MSi08L_iPSCs_undiff_BR5_N14_Unmixing_0_cmle.ome :: Channel:0
Channel 1 name: 01202024_MSi08L_iPSCs_undiff_BR5_N14_Unmixing_0_cmle.ome :: Channel:1
Channel 2 name: 01202024_MSi08L_iPSCs_undiff_BR5_N14_Unmixing_0_cmle.ome :: Channel:2
Channel 3 name: 01202024_MSi08L_iPSCs_undiff_BR5_N14_Unmixing_0_cmle.ome :: Channel:3
Channel 4 name: 01202024_MSi08L_iPSCs_undiff_BR5_N14_Unmixing_0_cmle.ome :: Channel:4
Channel 5 name: 01202024_MSi08L_iPSCs_undiff_BR5_N14_Unmixing_0_cmle.ome :: Channel:5
Channel 6 name: 01202024_MSi08L_iPSCs_undiff_BR5_N14_Unmixing_0_cmle.ome :: Channel:6
Channel 7 name: 01202024_MSi08L_iPSCs_undiff_BR5_N14_Unmixing_0_cmle.ome :: Channel:7
Channel 8 name: 01202024_MSi08L_iPSCs_undiff_BR5_N14_Unmixing_0_cmle.ome :: Channel:8
Channel 9 name: 01202024_MSi08L_iPSCs_undi

-----

## **EXTRACTION**

### **`STEP 1` - Create composite image**

&#x1F453; **FYI:** This code block creates the composite image of the organelle channels. The intensity of each channel is combined with specified weights.

#### &#x1F6D1; &#x270D; **User Input Required:**

Please specify the following values:
- `Invert_PM`: "True" inverts the intensities of the plasma membrane channel when added to the composite image; "False" leaves the intensity values as is when added to the composite image
- `PM_channel`: the index of the channel containing your plasma membrane label. Image indexing begins with 0, not 1. Reference the channel numbers indicated in the Napari window for easy reference.
- `weight_ch0`: the weight of channel 0 (the first channel); a value of 0 will exclude this channel from the compostite; large values will cause this channel to be more prominent in the final composite image
- `weight_ch1`: the weight of channel 1
- `weight_ch2`: the weight of channel 2
- `weight_ch3`: the weight of channel 3
- `weight_ch4`: the weight of channel 4
- `weight_ch5`: the weight of channel 5
- `weight_ch6`: the weight of channel 6
- `weight_ch7`: the weight of channel 7
- `weight_ch8`: the weight of channel 8
- `weight_ch9`: the weight of channel 9

In [11]:
#### USER INPUT REQUIRED ###
invert_pm = False
PM_CH = 7
w0 = 0
w1 = 0
w2 = 2
w3 = 2
w4 = 2
w5 = 2
w6 = 2
w7 = 0
w8 = 0
w9 = 0

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This code block creates the composite based on the settings above. The image is then added to Napari as a new layer for visual comparison to the input image. 

Use the Napari viewer to iteratively adjust the weights selected above.

In [12]:
# make aggregate
struct_img_raw = membrane_composite(img_data,
                                weight_ch0= w0,
                                weight_ch1= w1,
                                weight_ch2= w2,
                                weight_ch3= w3,
                                weight_ch4= w4,
                                weight_ch5= w5,
                                weight_ch6= w6,
                                weight_ch7= w7,
                                weight_ch8= w8,
                                weight_ch9= w9,
                                Invert_PM = invert_pm,
                                PM_Channel = PM_CH)

# adding image to Napari as a new layer
viewer.layers.clear()
viewer.grid.enabled = False
viewer.reset_view()
viewer.add_image(img_data, scale=scale)
viewer.add_image(struct_img_raw, scale=scale, name="1-Cell: Create composite")

<Image layer '1-Cell: Create composite' at 0x201094fb340>

### **`STEP 2` - Segment nuclei (from label)**
#### **`A. Select channel`**

#### &#x1F6D1; &#x270D; **User Input Required:**

Please specify which channel includes your nuclei label:
- `NUC_CH`: the index of the channel containing your nuclei label. Image indexing begins with 0, not 1. Reference the channel numbers indicated in the Napari window for easy reference.

In [13]:
#### USER INPUT REQUIRED ###
NUC_CH = 1

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This code block extracts the nuclei channel from your multi-channel image. It will be the only part of the image used in the rest of this section of the workflow. The single nuclei channel is added to the Napari viewer.

In [14]:
# select channel
raw_nuclei = select_channel_from_raw(img_data, NUC_CH)

# add single channel as a new layer
viewer.add_image(raw_nuclei, scale=scale, name="2-Nuc: Extract Channel")

<Image layer '2-Nuc: Extract Channel' at 0x2010f9c7b50>

### **`B. Rescale and smooth image`**

&#x1F453; **FYI:** This code block rescales the image so that the pixel/voxel with the highest intensity is set to 1 and the one with the lowest intensity is set to 0. The image is then *optionally* smoothed using a Gaussian and/or median filter. 

<mark> Include more information on the Gaussian and median filtering methods here </mark>

#### &#x1F6D1; &#x270D; **User Input Required:**

Please specify the amount of filter to use for each method. Higher values indicate more smoothing:
- `med_filter_size`: the size of the median filter to apply; if 0 is used, no filter will be applied
- `gaussian_smoothing_sigma`: the sigma to apply in the Gaussian filtering step; if 0 is used, no filter will be applied

In [15]:
#### USER INPUT REQUIRED ###
nuc_med_filter_size = 4   
nuc_gaussian_smoothing_sigma = 2.0

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This code block rescales the image and applies the specified median and Gaussian filters. The image is then added to Napari as a new layer for visual comparison to the input image. 

Use the Napari viewer to iteratively adjust the smoothing settings selected above.

In [16]:
# rescaling and smoothing input image
nuclei =  scale_and_smooth(raw_nuclei,
                           median_size = nuc_med_filter_size, 
                           gauss_sigma = nuc_gaussian_smoothing_sigma)

# adding image to Napari as a new layer
viewer.add_image(nuclei, scale=scale, name="2-Nuc: Rescale and Smooth")

<Image layer '2-Nuc: Rescale and Smooth' at 0x20106f040a0>

### **`C. Apply log transform and threshold`**

&#x1F453; **FYI:** This code block applies a log transform and creates semantic segmentation using [Li's Minimum Cross Entropy](https://scikit-image.org/docs/stable/api/skimage.filters.html#skimage.filters.threshold_li) method. `Semantic segmentation` is the process of deciding whether a pixel/voxel should be included in an object (labeled with a value of 1) or should be considered as part of the background (labeled with a value of 0). A semantic segmentation does not discern individual objects from one another.


#### &#x1F6D1; &#x270D; **User Input Required:**

Please specify the following information:
- `threshold_factor`: adjustment to make to the local threshold; larger values make the segmentation more stringent (less area included)
- `thresh_min`: minimum bound of the threshold
- `thresh_max`: maximum bound of the threshold

In [17]:
#### USER INPUT REQUIRED ###
nuc_threshold_factor = 0.9
nuc_thresh_min = .1
nuc_thresh_max = 1.

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This code block executes the log transform and threshold using the settings above.

Use the Napari viewer to iteratively adjust the filter settings as needed.

In [18]:
# log transform the image, calculate the threshold value using Li minimum cross entropy method, inverse log transform the value
li_thresholded = apply_log_li_threshold(nuclei, 
                                        thresh_factor=nuc_threshold_factor, 
                                        thresh_min=nuc_thresh_min, 
                                        thresh_max=nuc_thresh_max)

# adding image to Napari as a new layer
viewer.add_image(li_thresholded, scale=scale, name="2-Nuc: Threshold", opacity=0.3, colormap="cyan", blending='additive')

<Image layer '2-Nuc: Threshold' at 0x201106fb670>

### **`D. Remove small holes and objects`**

&#x1F453; **FYI:** This code block cleans up the semantic segmentation by filling small holes and/or removing small objects that can be considered errors in the initial segmentation. 

#### &#x1F6D1; &#x270D; **User Input Required:**

Please specify the following values:
- `hole_min_width`: the width of the smallest hole to be filled
- `hole_max_width`: the width of the largest hole to be filled
- `small_object_width`: the width of the largest object to be removed; any object smaller than this size will be removed
- `fill_filter_method`: "3D" processes the image taking into account segmentation values in three dimensions (XYZ); "slice-by-slice" processes each Z-slice in the image separately, not considering the segmentation results in higher or lower Z planes.

In [19]:
#### USER INPUT REQUIRED ###
nuc_hole_min_width = 0
nuc_hole_max_width = 40 

nuc_small_object_width = 25

nuc_fill_filter_method = "3D"

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This code block executes the dot filter using the settings above.

Use the Napari viewer to iteratively adjust the filter settings as needed. 

*Hint: white pixels/voxels are the ones remaining after this step*

In [20]:
# combine the above functions into one for downstream use in plugin
cleaned_img = fill_and_filter_linear_size(li_thresholded, 
                                           hole_min=nuc_hole_min_width, 
                                           hole_max=nuc_hole_max_width, 
                                           min_size=nuc_small_object_width,
                                           method=nuc_fill_filter_method)

# adding image to Napari as a new layer
viewer.add_image(cleaned_img, scale=scale, name="2-Nuc: Fill holes and remove small objects", colormap="magenta", blending="additive")

<Image layer '2-Nuc: Fill holes and remove small objects' at 0x20110be44c0>

### **`E. - Label objects`**

#### &#x1F3C3; **Run code; no user input required**
&#x1F453; **FYI:** This code block takes the semantic segmentation and creates an `instance segmentation`. In this output, each individual object in the image is given a unique ID number. The background pixels/voxels are still labeled as 0, but now each pixle/voxel within an object is labeled as a positive integer. 

In this workflow objects are separated based on connectivity: if a pixel/voxel is touching another pixel/voxel in any direction, they are considered the same object and each pixel/voxel within that object is labeled as the same unique ID number. 

*In the Napari viewer, the image is added as a "labels" layer where each object appears as a different color.*

In [21]:
# create instance segmentation based on connectivity
nuclei_labels = label_uint16(cleaned_img)

# adding image to Napari as a new layer
viewer.add_labels(nuclei_labels, scale=scale, name="2-Nuc: Instance segmentation")

<Labels layer '2-Nuc: Instance segmentation' at 0x20106cd1000>

-----
## **CORE-PROCESSING**

### **`STEP 3` - Watershed for cellmask**

&#x1F453; **FYI:** This code segments all of the cellmasks in the image. The process is accomplished with a [watershed](https://scikit-image.org/docs/stable/api/skimage.segmentation.html#skimage.segmentation.watershed) algorithm on the plasma membrane channel (values are inverted) where the nuclei are used as seeds. To perform the watershed in 3D, set Method = `"3D"`, or set Method = `"slice_by_slice"` to run it in 2D.

#### &#x1F6D1; &#x270D; **User Input Required:**

Please specify the following values:
- `PM_channel`: the index of the channel containing your plasma membrane label. Image indexing begins with 0, not 1. Reference the channel numbers indicated in the Napari window for easy reference.
- `Method`: "3D" processes the image taking into account segmentation values in three dimensions (XYZ); "slice-by-slice" processes each Z-slice in the image separately, not considering the segmentation results in higher or lower Z planes.


In [22]:
# parameters for watershed
PM_CH = 7
method = "3D"

In [23]:
# Segmentate multiple cellmasks via watershed
watershed_cell = invert_pm_watershed(img_data,
                                  nuclei_labels,
                                  PM_CH,
                                  method)                       
                                  
# adding image to Napari as a new layer
viewer.add_image(watershed_cell, scale=scale, name="3-Cell: Watershed")

<Image layer '3-Cell: Watershed' at 0x20126150910>

-----
## **POST-PROCESSING**

### **`STEP 4` - Select cell from watershed**

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** The cell and associated nucleus with the highest combined organelle intensity is then selected for downstream single-cell analysis.

In [24]:
# Select single cellmask based on maximum intensity in composite
raw_cellmask = choose_cell(struct_img_raw,
                           nuclei_labels,
                           watershed_cell)

# adding image to Napari as a new layer
viewer.add_image(raw_cellmask, scale=scale, name="4-Cell: Select Cell")

<Image layer '4-Cell: Select Cell' at 0x20118983670>

-----
## **POST-POST-PROCESSING**

### **`STEP 5` - Hole Filling**

&#x1F453; **FYI:** For some final cleaning, the [hole filling](https://github.com/AllenInstitute/aics-segmentation/blob/master/docs/API.md#hole_filling) function from Allen Cell [aicssegmentation](https://github.com/AllenCell/aics-segmentation) is used. To perform the hole filling in 3D, set fill_2d = `False`, or set fill_2d = `True` to run it in 2D. Hole Filling in 2D could be useful for removing topological tunnels aligned with the Z-axis, as the tunnel will become a series of holes when observed slice by slice

#### &#x1F6D1; &#x270D; **User Input Required:**

Please specify the following values:
- `hole_min`: the width of the smallest hole to be filled
- `hole_max`: the width of the largest hole to be filled
- `fill_2d`: whether to fill holes in the image slice by slice (True) or in 3D (False)

In [25]:
# parameters for hole filling
hole_min = 0
hole_max = 300
fill_2d = True

In [26]:
# fill holes in the cellmask
cellmask_obj = hole_filling(raw_cellmask,
                            hole_min,
                            hole_max,
                            fill_2d)

# adding image to Napari as a new layer
viewer.add_image(cellmask_obj, scale=scale, name="5-Cell: Fill holes in Cellmask")

<Image layer '5-Cell: Fill holes in Cellmask' at 0x20118cd3070>

-----
## **EXPORT**

### **`STEP 6` - Stack layers**

#### &#x1F3C3; **Run code; no user input required**
&#x1F453; **FYI:** This code block stacks the nucleus and cell into one multichannel image.

In [27]:
stack = stack_layers(nuclei_labels = nuclei_labels, cellmask = cellmask_obj)
print(f"Stack mask file structure: {np.shape(stack)}")
print(f"The dimension of '2' represents the 'nucleus' and 'cell' in that order.")

Stack mask file structure: (2, 35, 796, 796)
The dimension of '2' represents the 'nucleus' and 'cell' in that order.


-----
## **SAVING**

### **`Saving` - Save the segmentation output**

#### &#x1F3C3; **Run code; no user input required**
&#x1F453; **FYI:** This code block saves the instance segmentation output to the `out_data_path` specified earlier.

In [28]:
out_file_n = export_inferred_organelle(stack, "masks_D", meta_dict, out_data_path)

saved file: 01202024_MSi08L_iPSCs_undiff_BR5_N14_Unmixing_0_cmle.ome-masks_D


-----
-----
## **Define functions**
The following code includes an example of how the workflow steps above are combined into functions. The final combined `infer_masks_D` function can be run below to process a single image. It is included in the [batch process notebook](batch_process_segmentations.ipynb) to run the above analysis on multiple cells.

This function can utilized from infer-subc using:
```python
infer_subc.organelles.masks.infer_masks_D()
```

#### &#x1F3C3; **Run code; no user input required**
&#x1F453; **FYI:** This code block defines the `_infer_masks_masks_D()`, function. It is applied below.

In [29]:
##########################
#  _infer_nuclei
##########################
def _infer_nuclei_fromlabel(in_img: np.ndarray, 
                            nuc_ch: Union[int,None],
                            median_sz: int, 
                            gauss_sig: float,
                            thresh_factor: float,
                            thresh_min: float,
                            thresh_max: float,
                            min_hole_w: int,
                            max_hole_w: int,
                            small_obj_w: int,
                            fill_filter_method: str
                            ) -> np.ndarray:
    """
    Procedure to infer nuclei from linear unmixed input.

    Parameters
    ------------
    in_img: np.ndarray
        a 3d image containing all the channels
    nuc_ch:
        the index of the channel containing your nuclei label
    median_sz: int
        width of median filter for nuclei signal
    gauss_sig: float
        sigma for gaussian smoothing of  nuclei signal
    thresh_factor: float
        adjustment factor for log Li nuclei threholding
    thresh_min: float
        abs min threhold for log Li nuclei threholding
    thresh_max: float
        abs max threhold for log Li nuclei threholding
    min_hole_w: int
        hole filling minimum for nuclei post-processing
    max_hole_w: int
        hole filling cutoff for nuclei post-processing
    small_obj_w: int
        minimum object size cutoff for nuclei post-processing
    fill_filter_method:
        determines if fill and filter should be run 'sice-by-slice' or in '3D' (nucleus)

    Returns
    -------------
    nuclei_object
        mask defined extent of NU
    
    """
    ###################
    # EXTRACT
    ###################                
    nuclei = select_channel_from_raw(in_img, nuc_ch)

    ###################
    # PRE_PROCESSING
    ###################                
    nuclei =  scale_and_smooth(nuclei,
                        median_size = median_sz, 
                        gauss_sigma = gauss_sig)

    ###################
    # CORE_PROCESSING
    ###################
    nuclei_object = apply_log_li_threshold(nuclei, 
                                           thresh_factor=thresh_factor, 
                                           thresh_min=thresh_min, 
                                           thresh_max=thresh_max)

    ###################
    # POST_PROCESSING
    ###################
    nuclei_object = fill_and_filter_linear_size(nuclei_object, 
                                                hole_min=min_hole_w, 
                                                hole_max=max_hole_w, 
                                                min_size=small_obj_w,
                                                method=fill_filter_method)

    nuclei_labels = label_uint16(nuclei_object)

    return nuclei_labels

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This code block applies the function above to your test image. The settings specified above are applied here.

In [30]:
# run nuclei segmentation from label
_NU_objects = _infer_nuclei_fromlabel(img_data,
                                NUC_CH,
                                nuc_med_filter_size,
                                nuc_gaussian_smoothing_sigma,
                                nuc_threshold_factor,
                                nuc_thresh_min,
                                nuc_thresh_max,
                                nuc_hole_min_width,
                                nuc_hole_max_width,
                                nuc_small_object_width,
                                nuc_fill_filter_method)

# confirm this output matches the output saved above
print(f"The segmentation output here matches the output created above: {np.all(nuclei_labels == _NU_objects)}")

The segmentation output here matches the output created above: True


In [31]:
##########################
# _infer_cellmask_masks_D
# alternative workflow "d"
##########################
def _infer_cellmask_masks_D(in_img: np.ndarray,
                           pm_ch: Union[int,None],
                           weights: list[int],
                           invert_pm: bool,
                           nuclei_labels: np.ndarray,
                           cell_method: str,
                           hole_min: int,
                           hole_max: int,
                           fill_2d: bool):
    
    """
    Procedure to infer intermediate masks from linear unmixed input.

    Parameters
    ------------
    in_img: 
        a 3d image containing all the channels
    pm_ch:
        the index of the channel containing your plasma membrane label
    weights:
        a list of int that corresond to the weights for each channel in the composite; use 0 if a channel should not be included in the composite image
    invert_pm:
        True = invert plasma membrane channel in the composite
        False = do not invert plasma membrane channel in the composite
    nuclei_labels:
        a 3d image containing the inferred nuclei objects
    cell_method:
        The type of watershedding method to perform for the cellmask. Options include:
        "3D" (3D watershedding), "slice-by-slice" (2D watershedding).
    hole_min: 
        the minimum hole width to be filled in the cellmask
    hole_max: 
        the maximum hole width to be filled in the cellmask
    fill_2d:
        True = fill cellmask holes in 2D (slice by slice)
        False = fill cellmask holes in 3D
    
    Returns
    -------------
    cellmask:
        a logical/labels object defining boundaries of cellmask

    """

    ###################
    # EXTRACT
    ###################

    struct_img_raw = membrane_composite(in_img, *weights, invert_pm, pm_ch)

    ###################
    # CORE_PROCESSING
    ###################

    watershed_cell = invert_pm_watershed(in_img,
                                  nuclei_labels,
                                  pm_ch,
                                  cell_method)
    
    ###################
    # POST_PROCESSING
    ###################
    
    raw_cellmask = choose_cell(struct_img_raw,
                           nuclei_labels,
                           watershed_cell)
    
    #########################
    # POST- POST_PROCESSING
    #########################

    cellmask_obj = hole_filling(raw_cellmask,
                            hole_min,
                            hole_max,
                            fill_2d)
    
    return label_bool_as_uint16(cellmask_obj)
    

In [32]:
# run nuclei segmentation from label
_CM_object = _infer_cellmask_masks_D(img_data,
                                    PM_CH,
                                    [w0, w1, w2, w3, w4, w5, w6, w7, w8, w9],
                                    invert_pm,
                                    _NU_objects,
                                    method,
                                    hole_min,
                                    hole_max,
                                    fill_2d)

# confirm this output matches the output saved above
print(f"The segmentation output here matches the output created above: {np.all(cellmask_obj == _CM_object)}")

The segmentation output here matches the output created above: True


#### &#x1F3C3; **Run code; no user input required**
&#x1F453; **FYI:** This code block defines the `infer_masks_D()`, function, the complete function that combines the three above and that will be used for batch processing. It is applied below.

In [33]:
def _infer_masks_D(in_img: np.ndarray,
                   pm_ch: Union[int,None],
                   nuc_ch: Union[int,None],
                   weights: list[int],
                   median_sz: int, 
                   gauss_sig: float,
                   thresh_factor: float,
                   thresh_min: float,
                   thresh_max: float,
                   min_hole_w: int,
                   max_hole_w: int,
                   small_obj_w: int,
                   fill_filter_method: str,
                   invert_pm: bool,
                   cell_method: str,
                   hole_min: int,
                   hole_max: int,
                   fill_2d: bool):
    
    """
    Procedure to infer intermediate masks from linear unmixed input.

    Parameters
    ------------
    in_img: 
        a 3d image containing all the channels
    pm_ch:
        the index of the channel containing your plasma membrane label
    nuc_ch:
        the index of the channel containing your nuclei label
    weights:
        a list of int that corresond to the weights for each channel in the composite; use 0 if a channel should not be included in the composite image
    median_sz: int
        width of median filter for nuclei signal
    gauss_sig: float
        sigma for gaussian smoothing of nuclei signal
    thresh_factor: float
        adjustment factor for log Li nuclei threholding
    thresh_min: float
        abs min threhold for log Li nuclei threholding
    thresh_max: float
        abs max threhold for log Li nuclei threholding
    min_hole_w: int
        hole filling minimum for nuclei post-processing
    max_hole_w: int
        hole filling cutoff for nuclei post-processing
    small_obj_w: int
        minimum object size cutoff for nuclei post-processing
    fill_filter_method:
        determines if fill and filter should be run 'sice-by-slice' or in '3D' (nucleus)
    invert_pm:
        True = invert plasma membrane channel in the composite
        False = do not invert plasma membrane channel in the composite
    cell_method:
        The type of watershedding method to perform for the cellmask. Options include:
        "3D" (3D watershedding), "slice-by-slice" (2D watershedding).
    hole_min: 
        the minimum hole width to be filled in the cellmask
    hole_max: 
        the maximum hole width to be filled in the cellmask
    fill_2d:
        True = fill cellmask holes in 2D (slice by slice)
        False = fill cellmask holes in 3D
    
    Returns
    -------------
    mask_stack:
        a two channel np.ndarray constisting of the nucleus and cell (one object per channel)

    """

    ##########################
    # get nuclei labels #
    ##########################

    nuclei_labels = _infer_nuclei_fromlabel(in_img,
                                nuc_ch,
                                median_sz,
                                gauss_sig,
                                thresh_factor,
                                thresh_min,
                                thresh_max,
                                min_hole_w,
                                max_hole_w,
                                small_obj_w,
                                fill_filter_method)
    
    ##########################
    # get cellmask #
    ##########################

    cellmask_obj = _infer_cellmask_masks_D(in_img,
                                    pm_ch,
                                    weights,
                                    invert_pm,
                                    nuclei_labels,
                                    cell_method,
                                    hole_min,
                                    hole_max,
                                    fill_2d)
    
    ###################
    ### stack layers ##
    ###################

    stack = stack_layers(nuclei_labels = nuclei_labels,
                         cellmask = cellmask_obj)
    
    return stack

In [34]:
_stacked_masks = _infer_masks_D(img_data,
                                PM_CH,
                                NUC_CH,
                                [w0, w1, w2, w3, w4, w5, w6, w7, w8, w9],
                                nuc_med_filter_size,
                                nuc_gaussian_smoothing_sigma,
                                nuc_threshold_factor,
                                nuc_thresh_min,
                                nuc_thresh_max,
                                nuc_hole_min_width,
                                nuc_hole_max_width,
                                nuc_small_object_width,
                                nuc_fill_filter_method,
                                invert_pm,
                                method,
                                hole_min,
                                hole_max,
                                fill_2d)

#confirm this output matches the output saved above
print(f"The segmentation output here matches the output created above: {np.all(stack == _stacked_masks)}")

The segmentation output here matches the output created above: True


-------------
### ✅ **INFER MASKS COMPLETE!**

Now that a single cell has been identified for analysis, the organelle segmentions need to be completed. Continue on to complete the organelle notebooks (1.2 through 1.7) associated to each of the organelles you would like to include in your analysis:
- Infer [`lysosomes`](1.2_infer_lysosome.ipynb)
- Infer [`mitochondria`](1.3_infer_mitochondria.ipynb)
- Infer [`golgi`](1.4_infer_golgi.ipynb)
- Infer [`peroxisomes`](1.5_infer_peroxisome.ipynb)
- Infer [`endoplasmic reticulum (ER)`](1.6_infer_ER.ipynb)
- Infer [`lipid droplets`](1.7_infer_lipid_droplet.ipynb)